## Fetch text data

In [13]:
combined_df = pd.concat([pop_df, rap_df])
artists_df = combined_df[['spotify_id', 'name']].drop_duplicates()

artists_df['representative_text'] = ''

artists_df = artists_df.reset_index(drop=True)

print(f"Total count of unique pop and rap artists: {len(artists_df)}")
artists_df.head()

Total count of unique pop and rap artists: 5404


,spotify_id,name,representative_text
0,4Lm0pUvmisUHMdoky5ch2I,Apocalyptica,
1,6M2wZ9GZgrQXHCFfjv46we,Dua Lipa,
2,0SIXZXJCAhNU8sxK0qm7hn,Stefanie Sun,
3,7jXoGtR69J2iYCefc58MZX,Twins,
4,2sjnyyL9NXijL3Fr2eLikf,Aspova,


In [29]:
import lyricsgenius
import concurrent.futures
import time
import csv

genius = lyricsgenius.Genius("SQ5PQucPjwa_LEDaobBQyxewdh80ZB-KhaBau73yASMRNIvnijWprQSi6ZDnUb7N", timeout=10, retries=3, remove_section_headers=True)
test_artists = artists_df["name"].tolist()

In [8]:
def fetch_top3(name): 
    try: 
        artist = genius.search_artist(name, max_songs=3, sort="popularity")
        if artist: 
            lyrics = []
            for song in artist.songs:
                lyr = (song.lyrics).split("Lyrics")[1]
                lyrics.append(lyr)
            
            return {"Artist": name ,"Lyrics": lyrics}
        
    except Exception as e:
        print(f"Error fetching song by {name}: {e}")
        return None 

In [ ]:
for i in range(0,len(test_artists),4): 
    batch = test_artists[i: i+4]
    rows = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        future_to_song = {executor.submit(fetch_top3, artist): artist for artist in batch}
        for future in concurrent.futures.as_completed(future_to_song):
            result = future.result()
            if result:
                rows.append(result)
    #for name in batch: 
    #    rows.append(fetch_top3(name))
    
     # Append to file
    with open("Artist_text.csv", mode="a", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Artist", "Lyrics"])
        writer.writerows(rows)

Searching for songs by Juice...
Searching for songs by Yomi...


Searching for songs by Ray Dalton...

Searching for songs by Kiana Ledé...

Changing artist name to 'YOMi'
Changing artist name to 'Juice WRLD'
Song 1: "Unnskyld"
Song 1: "EX"
Song 2: "I Choose You"
Song 2: "Kom nærmere"
Song 3: "Fairplay"

Reached user-specified song limit (3).
Done. Found 3 songs.
Song 3: "Erfaring"

Reached user-specified song limit (3).
Done. Found 3 songs.
Song 1: "Lucid Dreams"
Song 1: "Manila"
Song 2: "In My Bones"
Song 2: "All Girls Are the Same"
Song 3: "ALL WE GOT"

Reached user-specified song limit (3).
Done. Found 3 songs.
Song 3: "Wishing Well"

Reached user-specified song limit (3).
Done. Found 3 songs.
Searching for songs by Phoebe Ryan...

Searching for songs by MKTO...

Searching for songs by Joe Stone...

Searching for songs by Agnes...

Changing artist name to 'AGNEZ MO'
Song 1: "Damn I Love You"
Song 1: "Classic"
Song 1: "Is It Really Love"
Song 2: "Karena Ku Sanggup"
Song 2: "American

: 

: 

In [114]:
rows

[{'Artist': 'Yomi',
  'Lyrics': ["\n\nJeg skulle hørt på deg først\nOg ikke på meg selv\nOg ikke på meg selv\nMen om det er til no' trøst\nSå er ingenting bestemt\nJa, ingenting er bestemt\nVet du føler at jeg svikta deg (Svikta deg)\nNår alt jeg prøvde var å vise vei (Vise vei)\nMen jeg tror jeg leste skiltet feil\n\nSå la meg bare si unnskyld, okei?\nHører du meg?\nHører du meg?\nHører du meg?\nJa, det er bare min skyld, okei?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\n\nDu skulle sett deg selv\nSånn jeg så deg først, sånn jeg så deg først\nDu så ut som en drøm\nHenda som en bønn\nMynter i en brønn\n\nSå la meg bare si unnskyld, okei?\nHører du meg?\nHører du meg?\nHører du meg?\nJa, det er bare min skyld, okei?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\nHører du meg?\n\nDet va’kke dagen min\n(Det va'kke dagen min\nDet va'kke dagen min)\nDet va’kke dagen min\n

In [137]:
df_tes = pd.read_csv("Artist_text.csv")
df_tes["Lyrics"] = df_tes["Lyrics"].apply(ast.literal_eval)